In [1]:
# Installing relevant libraries
!pip --quiet install transformers gradio

In [2]:
# Importing the libaries
import transformers
from transformers import pipeline
import gradio as gr

In [3]:
# Get a list of supported tasks, in the Huggingface pipelines
from transformers.pipelines import PIPELINE_REGISTRY

print(PIPELINE_REGISTRY.get_supported_tasks())

['audio-classification', 'automatic-speech-recognition', 'depth-estimation', 'document-question-answering', 'feature-extraction', 'fill-mask', 'image-classification', 'image-feature-extraction', 'image-segmentation', 'image-text-to-text', 'image-to-image', 'image-to-text', 'mask-generation', 'ner', 'object-detection', 'question-answering', 'sentiment-analysis', 'summarization', 'table-question-answering', 'text-classification', 'text-generation', 'text-to-audio', 'text-to-speech', 'text2text-generation', 'token-classification', 'translation', 'video-classification', 'visual-question-answering', 'vqa', 'zero-shot-audio-classification', 'zero-shot-classification', 'zero-shot-image-classification', 'zero-shot-object-detection']


In [4]:
# Some sample text to summarize to see if model works
text = "The Toronto Raptors have picked up the 2026-27 team option on Darko Rajakovic’s contract per Michael Grange. This move signifies a substantial vote of confidence in the second year head coach as they are locking him up for the next several seasons. Toronto is in the midst of a rebuild currently, and although the team’s record is not positive, and hasn’t been for a couple of seasons, there are clear indicators that Rajakovic is a more than capable coach. RJ Barrett playing the best basketball of his career in Toronto is a direct result of Rajakovic’s offensive scheme, which focuses on using motion to create easy and effective looks. Rajakovic’s defense scheme has also worked wonders this season, as Toronto has risen to 14th in defensive rating this year. His emphasis on pressuring the ball, while also having the backend rotations be cohesive have empowered this often injured Raptors team to be a stout defensive unit. This contract option pick up shows that Rajakovic has proved that he can be trusted to be the leader of this rebuild, and that the front office believes he will succeed in making Toronto better. With this season almost coming to a close, it will cap off Rajakovic’s second season as the head coach of the team."

In [5]:
# Print the model being used as default
summarizer = pipeline("summarization")

print(summarizer.model)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [6]:
# Importing the PEGASUS tokenizer to help with long text.
# Unlike BART, PEGASUS is pre-trained for abstractive(rewrites text in a new way) summarization and can handle longer documents more effectively. Can handle longer sequences (~1024-2048 tokens, depending on variant)
# Tokenization helps process large text efficiently, ensuring input sequences fit within the model's token limit.
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-large")

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Using the google/pegasus-large model, while sorting for top downloaded summarization models on the Hugging Face website. As it is highly effective for abstractive summarization tasks. The Pegasus model is pre-trained on a large-scale dataset and fine-tuned for a wide variety of summarization tasks, making it suitable for generating high-quality summaries.
summarize = pipeline("summarization", model="google/pegasus-large", tokenizer=tokenizer)

def summarization(text):
  summarization_results = summarize(text, max_length=300, min_length=30, do_sample=False)
  return summarization_results

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


In [8]:
# Summarizing the same text
print(summarize(text, max_length=250, min_length=30, do_sample=False))

[{'summary_text': 'Toronto is in the midst of a rebuild currently, and although the team’s record is not positive, and hasn’t been for a couple of seasons, there are clear indicators that Rajakovic is a more than capable coach. Rajakovic’s defense scheme has also worked wonders this season, as Toronto has risen to 14th in defensive rating this year. With this season almost coming to a close, it will cap off Rajakovic’s second season as the head coach of the team.'}]


In [9]:
# Creating the Gradio interface for the demo
demo = gr.Interface(
    fn=summarization,
    inputs=gr.Textbox(lines=2, placeholder="Enter text here..."),
    outputs="text")
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://75c76cbc3b5fca9aff.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
